In [2]:
import pandas as pd
import numpy as np
import aiohttp
import asyncio

MAX_TRACES_PER_USER = 5
DESIRED_TRACES = 1000
OUTPUT_FOLDER = "output"

In [28]:
data = pd.read_csv(
    "web_routineness_data/browsing.csv",
    dtype={"id": int, "web_visits_id": int},
    parse_dates=["used_at"],
).drop(columns=["id"])
data.head()

,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain
0,111761051,1421,0,2018-10-05 22:02:38,4,ebesucher.de,NaN
1,111761051,1421,0,2018-10-05 21:41:22,10,ebesucher.de,NaN
2,111761051,1421,0,2018-10-05 21:45:36,5,ebesucher.de,NaN
3,111761051,1421,0,2018-10-05 21:42:22,181,ebesucher.de,NaN
4,111761055,1421,1,2018-10-05 21:47:57,2,klamm.de,NaN


In [29]:
# Sanity check: only keep panelists with a good number of web page visits
REQUIRED_DATAPOINTS = 200

panelists_to_filter = data.groupby("panelist_id").filter(
    lambda x: len(x) < REQUIRED_DATAPOINTS
)["panelist_id"].unique()

print(f"Filtering out {len(panelists_to_filter)} panelists with less than {REQUIRED_DATAPOINTS} datapoints")

data_filtered = data[~data["panelist_id"].isin(panelists_to_filter)].copy()
print("Rows:", len(data_filtered))
data_filtered.sort_values("used_at").iloc[np.r_[0:5, -5:0]]

Unclosed connection
client_connection: Connection<ConnectionKey(host='www.tchibo.de', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='www.g2play.net', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='news.google.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None)>


Filtering out 319 panelists with less than 200 datapoints
Rows: 9127431


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain
1369019,114016772,1849,498963,2018-10-01 00:00:00,8,nicequest.com,NaN
1315961,113973380,750,481401,2018-10-01 00:00:00,2,vasilisaroom.site,NaN
1232915,113894036,957,454324,2018-10-01 00:00:00,6,google.com,NaN
1520755,114158144,1982,552480,2018-10-01 00:00:00,2,facebook.com,NaN
1346500,113996676,1170,491134,2018-10-01 00:00:01,5,ebay-kleinanzeigen.de,NaN
8884856,124428860,1627,2918269,2018-10-31 23:59:58,16,ebay.de,NaN
8944905,124539076,1067,241123,2018-10-31 23:59:58,10,gfk.de,ecpo.
9091905,124804680,1226,813332,2018-10-31 23:59:58,452,mypersonalroutine.com,de.
8938760,124531492,177,2934742,2018-10-31 23:59:59,6,crazyslip.com,NaN
8984063,124604448,248,10894,2018-10-31 23:59:59,4,wish.com,NaN


In [30]:
# split data into 30 min buckets
BUCKET_SIZE = 30 * 60

start_time = data_filtered['used_at'].min()
buckets = data_filtered["used_at"].sub(start_time)
data_filtered['bucket'] = (buckets.dt.total_seconds() // BUCKET_SIZE).astype(int)
print("Rows:", len(data_filtered))
data_filtered.sort_values("used_at").iloc[np.r_[0:5, -5:0]]

Rows: 9127431


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain,bucket
1369019,114016772,1849,498963,2018-10-01 00:00:00,8,nicequest.com,NaN,0
1315961,113973380,750,481401,2018-10-01 00:00:00,2,vasilisaroom.site,NaN,0
1232915,113894036,957,454324,2018-10-01 00:00:00,6,google.com,NaN,0
1520755,114158144,1982,552480,2018-10-01 00:00:00,2,facebook.com,NaN,0
1346500,113996676,1170,491134,2018-10-01 00:00:01,5,ebay-kleinanzeigen.de,NaN,0
8884856,124428860,1627,2918269,2018-10-31 23:59:58,16,ebay.de,NaN,1487
8944905,124539076,1067,241123,2018-10-31 23:59:58,10,gfk.de,ecpo.,1487
9091905,124804680,1226,813332,2018-10-31 23:59:58,452,mypersonalroutine.com,de.,1487
8938760,124531492,177,2934742,2018-10-31 23:59:59,6,crazyslip.com,NaN,1487
8984063,124604448,248,10894,2018-10-31 23:59:59,4,wish.com,NaN,1487


In [31]:
# Filter out buckets whose size is outside the 35th-85th percentiles range
LOWER_BOUND = 0.35
UPPER_BOUND = 0.85

bucket_sizes = data_filtered.groupby(["panelist_id", "bucket"]).size()
lower_bound = bucket_sizes.quantile(LOWER_BOUND)
upper_bound = bucket_sizes.quantile(UPPER_BOUND)
print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")

data_good_buckets = data_filtered.groupby(["panelist_id", "bucket"]).filter(
    lambda x: lower_bound <= len(x) <= upper_bound
)
print("Rows:", len(data_good_buckets))
data_good_buckets.iloc[np.r_[0:5, -5:0]]

Lower bound: 8.0, Upper bound: 47.0
Rows: 4069727


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain,bucket
1,111761051,1421,0,2018-10-05 21:41:22,10,ebesucher.de,NaN,235
2,111761051,1421,0,2018-10-05 21:45:36,5,ebesucher.de,NaN,235
3,111761051,1421,0,2018-10-05 21:42:22,181,ebesucher.de,NaN,235
4,111761055,1421,1,2018-10-05 21:47:57,2,klamm.de,NaN,235
5,111761055,1421,2,2018-10-05 21:47:40,1,klamm.de,NaN,235
9151238,124906196,1739,236,2018-10-31 16:52:12,105,facebook.com,NaN,1473
9151239,124906196,1739,236,2018-10-31 16:51:04,8,facebook.com,NaN,1473
9151240,124906200,1739,2997793,2018-10-31 16:50:10,2,goo.gl,NaN,1473
9151241,124906204,1739,2997794,2018-10-31 16:50:12,52,focus.de,NaN,1473
9151242,124906208,1739,2997795,2018-10-31 16:51:12,60,berliner-sonntagsblatt.de,NaN,1473


In [32]:
# Filter out buckets whose web page visits all occur within a 15 minute range
# We want traces to be at least somewhat spread out in time
REQUIRED_TIME_SPAN = 15 * 60

data_good_range = data_good_buckets.groupby(["panelist_id", "bucket"]).filter(
    lambda x: (x["used_at"].max() - x["used_at"].min()).total_seconds() >= REQUIRED_TIME_SPAN
)
print("Rows:", len(data_good_range))
data_good_range.iloc[np.r_[0:5, -5:0]]

Rows: 2673105


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain,bucket
75,111762035,789,21,2018-10-05 16:53:08,4,amazon.de,NaN,225
82,111762043,789,27,2018-10-05 16:42:14,10,icloud.com,NaN,225
83,111762043,789,28,2018-10-05 16:31:04,2,icloud.com,NaN,225
84,111762043,789,29,2018-10-05 16:53:18,20,icloud.com,NaN,225
85,111762043,789,30,2018-10-05 16:52:49,5,icloud.com,NaN,225
9151238,124906196,1739,236,2018-10-31 16:52:12,105,facebook.com,NaN,1473
9151239,124906196,1739,236,2018-10-31 16:51:04,8,facebook.com,NaN,1473
9151240,124906200,1739,2997793,2018-10-31 16:50:10,2,goo.gl,NaN,1473
9151241,124906204,1739,2997794,2018-10-31 16:50:12,52,focus.de,NaN,1473
9151242,124906208,1739,2997795,2018-10-31 16:51:12,60,berliner-sonntagsblatt.de,NaN,1473


In [33]:
# Keep up to MAX_TRACES_PER_USER buckets per panelist


def filter_buckets(df):
    grouped = df.groupby("bucket")
    if len(grouped) <= MAX_TRACES_PER_USER:
        return df
    else:
        indices = np.arange(grouped.ngroups)
        np.random.shuffle(indices)
        selected_buckets = indices[:MAX_TRACES_PER_USER]
        return df[grouped.ngroup().isin(selected_buckets)]


data_pruned = (
    data_good_range.groupby("panelist_id")
    .apply(filter_buckets, include_groups=False)
    .reset_index()
    .drop(columns=["level_1"])
)
print("Rows:", len(data_pruned))
data_pruned.iloc[np.r_[0:5, -5:0]]

Rows: 216119


,panelist_id,web_visits_id,url,used_at,active_seconds,domain,subdomain,bucket
0,0,116529520,1336924,2018-10-07 14:14:34,8,youtube.com,NaN,316
1,0,116529520,1336925,2018-10-07 14:05:55,71,youtube.com,NaN,316
2,0,116529520,1336928,2018-10-07 14:20:55,300,youtube.com,NaN,316
3,0,116529520,248953,2018-10-07 14:00:19,9,youtube.com,NaN,316
4,0,116529520,1336928,2018-10-07 14:00:55,59,youtube.com,NaN,316
216114,2147,124905620,2997657,2018-10-31 14:44:56,8,meinestadt.de,veranstaltungen.,1469
216115,2147,124905620,2997658,2018-10-31 14:45:10,30,meinestadt.de,veranstaltungen.,1469
216116,2147,124905620,2997657,2018-10-31 14:44:33,5,meinestadt.de,veranstaltungen.,1469
216117,2147,124905620,2997659,2018-10-31 14:44:52,4,meinestadt.de,veranstaltungen.,1469
216118,2147,124905620,2997657,2018-10-31 14:41:18,185,meinestadt.de,veranstaltungen.,1469


In [34]:
# Combine subdomain and domain into a single column
data_transformed = data_pruned.copy()
data_transformed["subdomain"] = data_transformed["subdomain"].fillna("")
data_transformed["domain"] = data_transformed["subdomain"] + data_transformed["domain"]
data_transformed.drop(columns=["subdomain"], inplace=True)
data_transformed.iloc[np.r_[0:5, -5:0]]

,panelist_id,web_visits_id,url,used_at,active_seconds,domain,bucket
0,0,116529520,1336924,2018-10-07 14:14:34,8,youtube.com,316
1,0,116529520,1336925,2018-10-07 14:05:55,71,youtube.com,316
2,0,116529520,1336928,2018-10-07 14:20:55,300,youtube.com,316
3,0,116529520,248953,2018-10-07 14:00:19,9,youtube.com,316
4,0,116529520,1336928,2018-10-07 14:00:55,59,youtube.com,316
216114,2147,124905620,2997657,2018-10-31 14:44:56,8,veranstaltungen.meinestadt.de,1469
216115,2147,124905620,2997658,2018-10-31 14:45:10,30,veranstaltungen.meinestadt.de,1469
216116,2147,124905620,2997657,2018-10-31 14:44:33,5,veranstaltungen.meinestadt.de,1469
216117,2147,124905620,2997659,2018-10-31 14:44:52,4,veranstaltungen.meinestadt.de,1469
216118,2147,124905620,2997657,2018-10-31 14:41:18,185,veranstaltungen.meinestadt.de,1469


In [ ]:
BATCH_SIZE = 100
remap = {}


def simplify_domain(domain):
    # some domains include a subdomain that don't response to GET requests
    # we should retry them with their main domain
    parts = domain.strip(".").split(".")
    if len(parts) > 2:
        return ".".join(parts[-2:])
    return None


async def build_mapping(domain, session):
    if domain not in remap:
        # prevent concurrent calls from both building the same mapping
        event = asyncio.Event()
        remap[domain] = event

        try:
            print(f"Fetching mapping for {domain}")
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:140.0) Gecko/20100101 Firefox/140.0",
            }
            if "facebook" in domain:
                # Facebook returns 400 error when we fake our User-Agent
                headers = None
            response = await session.get(f"http://{domain}", headers=headers)
            remap[domain] = response.url
            print(f"Mapping {domain} to {response.url}")
        except Exception as e:
            if str(e):
                print(f"Error fetching {domain}: {e}")
            else:
                print(f"Error fetching {domain}: {repr(e)}")

            simplified = simplify_domain(domain)
            if not simplified:
                print(f"Can't simplify {domain}, setting to None")
                remap[domain] = None
            else:
                await build_mapping(simplified, session)
                remap[domain] = remap[simplified]
                print(f"Simplified mapping {domain} to {remap[domain]}")

        event.set()
    elif isinstance(remap[domain], asyncio.Event):
        # caller expects this function to finish only after the mapping is built
        await remap[domain].wait()


async def build_all_mappings(unique_domains):
    i = 0
    while i < len(unique_domains):
        async with aiohttp.ClientSession(
            connector=aiohttp.TCPConnector(
                ssl=False,
                keepalive_timeout=2,
                limit=BATCH_SIZE * 1.5,
            ),
            raise_for_status=True,
            auto_decompress=False,
            timeout=aiohttp.ClientTimeout(total=45, sock_connect=10, sock_read=10),
        ) as session:
            end = min(i + BATCH_SIZE, len(unique_domains))
            await asyncio.gather(*(build_mapping(domain, session) for domain in unique_domains[i:end]))
            i += BATCH_SIZE
            print(f"{i / len(unique_domains) * 100:.2f}% done ({i}/{len(unique_domains)})")
        await asyncio.sleep(5)
    return remap

In [ ]:
data_remapped = data_transformed.copy()
unique_domains = data_transformed["domain"].unique()
mapping =  await build_all_mappings(unique_domains)
data_remapped["domain"] = data_remapped["domain"].map(mapping)

In [60]:
print("Rows:", len(data_pruned))
data_remapped.iloc[np.r_[0:5, -5:0]]

Rows: 216119


,panelist_id,web_visits_id,url,used_at,active_seconds,domain,bucket
0,0,116529520,1336924,2018-10-07 14:14:34,8,https://www.youtube.com/,316
1,0,116529520,1336925,2018-10-07 14:05:55,71,https://www.youtube.com/,316
2,0,116529520,1336928,2018-10-07 14:20:55,300,https://www.youtube.com/,316
3,0,116529520,248953,2018-10-07 14:00:19,9,https://www.youtube.com/,316
4,0,116529520,1336928,2018-10-07 14:00:55,59,https://www.youtube.com/,316
216114,2147,124905620,2997657,2018-10-31 14:44:56,8,None,1469
216115,2147,124905620,2997658,2018-10-31 14:45:10,30,None,1469
216116,2147,124905620,2997657,2018-10-31 14:44:33,5,None,1469
216117,2147,124905620,2997659,2018-10-31 14:44:52,4,None,1469
216118,2147,124905620,2997657,2018-10-31 14:41:18,185,None,1469


In [63]:
# Filter out all traces with website visits to invalid URLs
data_good_urls = data_remapped.groupby(["panelist_id", "bucket"]).filter(
    lambda x: x["domain"].notna().all()
)
print(f"Rows: {len(data_good_urls)}")
print(f"Number of possible traces: {len(data_good_urls.groupby(['panelist_id', 'bucket']))}")
data_good_urls.iloc[np.r_[0:5, -5:0]]

Rows: 126761
Number of possible traces: 5555


,panelist_id,web_visits_id,url,used_at,active_seconds,domain,bucket
0,0,116529520,1336924,2018-10-07 14:14:34,8,https://www.youtube.com/,316
1,0,116529520,1336925,2018-10-07 14:05:55,71,https://www.youtube.com/,316
2,0,116529520,1336928,2018-10-07 14:20:55,300,https://www.youtube.com/,316
3,0,116529520,248953,2018-10-07 14:00:19,9,https://www.youtube.com/,316
4,0,116529520,1336928,2018-10-07 14:00:55,59,https://www.youtube.com/,316
216088,2147,122893736,2644857,2018-10-26 09:15:23,2,https://pixabay.com/,1218
216089,2147,122893736,113789,2018-10-26 09:15:35,4,https://pixabay.com/,1218
216090,2147,122893736,216905,2018-10-26 09:15:13,10,https://pixabay.com/,1218
216091,2147,122893736,216905,2018-10-26 09:15:25,10,https://pixabay.com/,1218
216092,2147,122893736,2644858,2018-10-26 09:16:05,26,https://pixabay.com/,1218


In [64]:
data_good_urls.to_csv(f"{OUTPUT_FOLDER}/cleaned_buckets.csv", index=False)

In [5]:
data_good_urls = pd.read_csv(
    f"{OUTPUT_FOLDER}/cleaned_buckets.csv",
    dtype={"id": int, "web_visits_id": int},
    parse_dates=["used_at"],
)